In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
 
# Load CSV file
def load_csv(file_path):
    data = pd.read_csv('incident_event_log.csv')
    return data


In [6]:
load_csv()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119993,INC0029231,Resolved,True,0,0,8,True,Caller 2321,Opened by 17,13/5/2016 11:46,...,True,Do Not Notify,?,?,?,?,code 7,Resolved by 11,13/5/2016 12:15,18/5/2016 13:07
119994,INC0029231,Closed,False,0,0,9,True,Caller 2321,Opened by 17,13/5/2016 11:46,...,True,Do Not Notify,?,?,?,?,code 7,Resolved by 11,13/5/2016 12:15,18/5/2016 13:07
119995,INC0029232,Resolved,True,0,0,0,True,Caller 382,Opened by 108,13/5/2016 11:48,...,False,Do Not Notify,?,?,?,?,code 6,Resolved by 52,?,18/5/2016 12:07
119996,INC0029232,Closed,False,0,0,1,True,Caller 382,Opened by 108,13/5/2016 11:48,...,False,Do Not Notify,?,?,?,?,code 6,Resolved by 52,?,18/5/2016 12:07


In [8]:

# Preprocess text
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize
    tokens = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens
 
# Train Word2Vec model
def train_word2vec(sentences):
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    return model
 
# Compute sentence embeddings by averaging word vectors
def compute_sentence_embeddings(sentences, model):
    embeddings = []
    for sentence in sentences:
        vectors = [model.wv[word] for word in sentence if word in model.wv]
        if vectors:
            sentence_embedding = np.mean(vectors, axis=0)
        else:
            sentence_embedding = np.zeros(model.vector_size)
        embeddings.append(sentence_embedding)
    return np.array(embeddings)
 
# Summarize text
def summarize_text(data, num_sentences=5):
    sentences = data['text'].tolist()
    preprocessed_sentences = [preprocess_text(sentence) for sentence in sentences]
    
    word2vec_model = train_word2vec(preprocessed_sentences)
    sentence_embeddings = compute_sentence_embeddings(preprocessed_sentences, word2vec_model)
    
    cosine_sim_matrix = cosine_similarity(sentence_embeddings)
    scores = cosine_sim_matrix.sum(axis=1)
    
    top_sentence_indices = scores.argsort()[-num_sentences:][::-1]
    summary = [sentences[i] for i in top_sentence_indices]
    return summary
 
# Main function
def main(file_path):
    data = pd.read_csv('incident_event_log.csv')
    summary = summarize_text(data)
    for sentence in summary:
        print(sentence)
 
# Example usage
file_path = "incident_event_log.csv"
main(file_path)

KeyError: 'text'